In [2]:
#Plotting map

import geopandas as gpd
import geoplot
import pandas as pd

# targetFiles = 'HealthyFood'
# targetFiles = 'UnhealthyFood'
# targetFiles = 'GenericTweets'
targetFiles = 'smallData'

RAW_CSV_DIR = f'./data/CSVFiles/'
PROCESSED_LOCATION_CSV_FILE = f'./{RAW_CSV_DIR}/{targetFiles}-LocationsDataframe.csv'

ModuleNotFoundError: No module named 'geoplot'

In [5]:
import sys
'geopandas' in sys.modules

False

In [ ]:
world = gpd.read_file(
    gpd.datasets.get_path('naturalearth_lowres')
)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(world.name)

In [ ]:
#TODO: Get full dataframe
df = pd.read_csv(PROCESSED_LOCATION_CSV_FILE)
df.head()

In [ ]:
df.Country.value_counts().head(35)

In [ ]:
#Translate country names top 35 countries

df.loc[df['Country'] == "México", 'Country'] = 'Mexico'
df.loc[df['Country'] == "España", 'Country'] = 'Spain'
df.loc[df['Country'] == "United States", 'Country'] = 'United States of America'
df.loc[df['Country'] == "Estados Unidos", 'Country'] = 'United States of America'
df.loc[df['Country'] == "Panamá", 'Country'] = 'Panama'
df.loc[df['Country'] == "Dominican Republic", 'Country'] = 'Dominican Rep.'
df.loc[df['Country'] == "Italia", 'Country'] = 'Italy'
df.loc[df['Country'] == "Alemania", 'Country'] = 'Germany'
df.loc[df['Country'] == "Espanya", 'Country'] = 'Spain'
df.loc[df['Country'] == "Republic of the Philippines", 'Country'] = 'Philippines'
df.loc[df['Country'] == "Reino unido", 'Country'] = 'United Kingdom'
df.loc[df['Country'] == "Suiza", 'Country'] = 'Switzerland'
df.loc[df['Country'] == "Francia", 'Country'] = 'France'
df.loc[df['Country'] == "Canadá", 'Country'] = 'Canada'





top_countries = df.Country.head(30)

In [ ]:
df = df[df.Country.isin(top_countries)]
df

In [ ]:
df.Country.value_counts().head(35)

In [ ]:
tweetsInCountry = []
n = 0
for i in world['name']:
    for j in df['Country']:
        if i == j:
            n = n + 1
    tweetsInCountry.append(n)
    n = 0


In [ ]:
tweetsInCountry

In [ ]:
import matplotlib.pyplot as plt
world['tweets'] = tweetsInCountry
plt.show(world.plot(column='tweets', cmap='OrRd', figsize=(15, 8)))
for i in range(len(world['tweets'])):
    print(world['name'][i])
    print(world['tweets'][i])

In [ ]:
fig, ax = plt.subplots(1, 1)

world.plot(column='tweets', ax=ax, legend=True);


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1)

divider = make_axes_locatable(ax)

cax = divider.append_axes("right", size="5%", pad=0.1)

world.plot(column='tweets', ax=ax, legend=True, cax=cax)

In [ ]:
fig, ax = plt.subplots(1, 1)

world.plot(column='tweets',
            ax=ax,
            legend=True,
            legend_kwds={'label': "Population by Country",
                         'orientation': "horizontal"})

In [ ]:
fig, ax = plt.subplots(1, 1)

plt.show(world.plot(column='tweets',
            ax=ax,
            legend=True,
            cmap='OrRd',
            figsize=(15, 8),
            legend_kwds={'label': "Tweets per Country",
                         'orientation': "horizontal"}))

#plt.savefig('test.png')


In [ ]:


world.plot(column='tweets', 
                    cmap='OrRd', legend=True,figsize=(12, 12), legend_kwds={'label': "Tweets per Country",
                         'orientation': "horizontal"})

In [ ]:
df_spain = df = df[df.Country == 'Spain']
df_spain = df_spain.reset_index()

df_spain

In [ ]:
# Cargar la capa temática
natalidad = "datos/natalidad.geojson"
map_data = gpd.read_file(natalidad)
map_data.head()

In [ ]:
df_spain.Coordinates[0]

In [ ]:
from shapely.geometry import Point

point_coordinates = []

for index, row in df_spain.iterrows():
    a = (float(i) for i in row['Coordinates'].replace('(', '').replace(')', '').replace(' ','').split(','))
    point = Point(a)
    point_coordinates.append(point)


In [ ]:
map_data.geometry[0].contains(point)


In [ ]:
tweetsInProvince = []
n = 0
for province in map_data['geometry']:
    for point in point_coordinates:
        if province.contains(point):
            n = n + 1
    tweetsInProvince.append(n)
    n = 0

In [ ]:
map_data['Tweets'] = tweetsInProvince
map_data

In [ ]:
population = []

for index, row in map_data.iterrows():
    
    print('{}: '.format(row['NAME_2']))
    val = int(input("Poblacion: "))
    population.append(val) 

map_data['Population']= population
map_data

In [ ]:
map_data.to_file("spain.geojson", driver='GeoJSON')


In [ ]:
x = gpd.read_file('spain.geojson')
x

In [ ]:
map_data['Population'] = x['Population']


In [ ]:
tweetDensity = map_data.Tweets/map_data.Population
map_data['TweetDensity'] = tweetDensity
map_data

In [ ]:
## Control del tamaño de la figura del mapa
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
 
# Control del encuadre (área geográfica) del mapa
ax.axis([-12, 5, 32, 48])
 
# Control del título y los ejes
ax.set_title('Densidad de tweets por Provincias en España', 
             pad = 20, 
             fontdict={'fontsize':20, 'color': '#4873ab'})
ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')
 
# Añadir la leyenda separada del mapa
from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.2)
 
# Generar y cargar el mapa
map_data.plot(column='TweetDensity', cmap='plasma', ax=ax,
              legend=True, cax=cax, zorder=5)
 
# Cargar un mapa base con contornos de países
oceanos = "datos/ne_50m_ocean.shx"
map_oceanos = gpd.read_file(oceanos)
map_oceanos.plot(ax=ax, color='#89c0e8', zorder=0)

In [ ]:
## Control del tamaño de la figura del mapa
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
 
# Control del encuadre (área geográfica) del mapa
ax.axis([-12, 5, 32, 48])
 
# Control del título y los ejes
ax.set_title('Tweets por Provincias en España', 
             pad = 20, 
             fontdict={'fontsize':20, 'color': '#4873ab'})
ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')
 
# Añadir la leyenda separada del mapa
from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.2)
 
# Generar y cargar el mapa
map_data.plot(column='Tweets', cmap='plasma', ax=ax,
              legend=True, cax=cax, zorder=5)
 
# Cargar un mapa base con contornos de países
oceanos = "datos/ne_50m_ocean.shx"
map_oceanos = gpd.read_file(oceanos)
map_oceanos.plot(ax=ax, color='#89c0e8', zorder=0)